In [52]:
import sys
import pathlib
import os
#sys.path.append("C:\\Users\\mnowatz\\Documents\\Dev\\aepe")
#print(sys.path)
import psycopg2
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
import database as db
from xml.etree.ElementTree import ElementTree, Element, SubElement
import io
import json
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import apsim.apsim.wrapper as apsim
from apsim.apsim.daymet import Weather
from apsim.apsim_input_writer import get_date, add_management_year
from apsim import run_apsim

In [3]:
dbconn = db.connect_to_db('database.ini')
raccoon_2018 = 'raccoon.raccoon_clu_ssurgo_2018'
#raccoon = pd.read_sql(f"SELECT * FROM {raccoon_2018} LIMIT 200;", dbconn)
greene = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE County='Greene';", dbconn)

In [4]:
spin_up_corn = json.loads( open( 'crop_jsons/maize.json', 'r' ).read() )
spin_up_soybean = json.loads( open( 'crop_jsons/soybean.json', 'r' ).read() )

### Step 1: Get the counties we are interested in.

In [5]:
'''
Returns a list of all unique entries in a column.

Args:
    dbconn {database connection} -- connection to postgresql database
    table {str} -- table name
    id_column (str) -- column of interest.
Returns:
    list of all unique entries in a table column
'''
def get_distinct(dbconn, table, id_column):
    entries = pd.read_sql(f'SELECT DISTINCT {id_column} FROM {table};', dbconn)
    entries = entries[id_column].tolist()
    return entries

In [6]:
#bv_query = 'SELECT * FROM raccoon.raccoon_clu_ssurgo_2018 WHERE fips = \'IA021\';'
'''Get info for a county of interest from a geopandas df
Args:
    dbconn {database connection} -- connection to postgresql database
    table {str} -- name of geopd table
    fips {str} -- fips id of the desired county eg. 'IA021'
    geom {str} -- column name that contains shape geometry
Returns:
    geopandas dataframe with county info
'''
def get_county(dbconn, table, fips, geom, limit=False, limit_num=100):
    #Get watershed as geopandas df
    if limit:
        query = f'SELECT * FROM {table} WHERE fips = \'{fips}\' LIMIT {limit_num};'
    else:
        query = f'SELECT * FROM {table} WHERE fips = \'{fips}\';'
    county_gpd = gpd.read_postgis(query, dbconn, geom_col=geom)
    return county_gpd

In [7]:
#buena_vista = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA021', "wkb_geometry")

### Step 2: Get the county centroid for creating APSIM met files

In [8]:
'''
Find and return the centroid of a geopandas geometry

Args: 
    geodf {dataframe} -- geopandas dataframe
    id {string} -- the id of interest in the geodf (e.g., 'fips' for county column)
    geometry (string) - geopd column with geometries

Returns:
    {np.array} -- lat and longitude of geometry
'''
def get_centroid(geodf, id, geometry):
    #get the geometry of interest by id - 'fips' for a county
    geom = geodf[[id, geometry]]
    #dissolve geometries to make one big geometry
    dissolved_geom = geom.dissolve(by=id)
    #find the centroid of the dissolved geometry and return its long, lat
    centroid = dissolved_geom[geometry].centroid
    coords = np.vstack([centroid.x, centroid.y]).T
    #change to array and lat, long
    centroid_coords = np.array([coords[0][1], coords[0][0]])
    return centroid_coords

In [9]:
#bv_centroid = get_centroid(buena_vista, 'fips', "wkb_geometry")

In [10]:
'''
Get the weather for given centroid and write to a .met file

Args:
    lat {float} -- latitude of centroid
    long {float} -- longitude of centroid
    year_star {int} -- starting year of weather data
    year_end {int} -- ending year of weather data
    path {str} -- path to write the met files
    filename {str} -- name to give the .met file

Returns:
    None
'''
def create_met(lat, long, start_year, end_year, filename, path='apsim_files/met_files'):
    weather_obj = Weather().from_daymet(lat, long, start_year, end_year)
    weather_obj.write_met_file(f'{path}/{filename}.met')

In [11]:
greene = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA073', "wkb_geometry")
greene_centroid = get_centroid(greene, 'fips', "wkb_geometry")

In [15]:
greene = Weather().from_daymet(greene_centroid[0], greene_centroid[1], 1980, 2019)

In [54]:
create_met(greene_centroid[0], greene_centroid[1], 1980, 2019, 'greene')

ParserError: Passed header=6 but only 1 lines in file

In [24]:
greene_df = greene.data
type(greene_df)

pandas.core.frame.DataFrame

In [48]:
#greene_df.to_excel('greene.xlsx', index=False)
wb = openpyxl.Workbook()
ws = wb.active
for r in dataframe_to_rows(green_df, index=False, header=True):
    ws.append(r)

In [49]:
ws.insert_rows(1)
ws['A1'] = 'This is a test to insert some text.'
wb.save("test.prn")

In [51]:
os.rename('test.prn', 'test.met')

In [55]:
greene_weather = create_met(greene_centroid[0], greene_centroid[1], 1980, 2019, 'greene')

In [ ]:
"""
Create all met files for each county in a given geometry.

Args:
    dbconn {database connection} -- connection to postgresql database
    counties {array/list} -- every county to be run on in the geometry
    table {str} -- name of the table within the database to get geometries from
    id_column {str} -- table column that has unique ids (in this case fips) for each county.
    geo_col {str} -- table column that has geometry.
    name_col {str} -- column that has the name for each county - can just use fips columns if no county names in table.

Returns:
    Met file for each county in counties.
"""
def create_all_met(dbconn, counties, table, id_col='fips', geo_col='wkb_geometry', name_col='county'):
    for i in counties:
        county = get_county(dbconn, table, i, geo_col)
        county_name = county[name_col][0].replace(" ", "_")
        print(f'Geopandas table for {county_name} county created.')
        centroid = get_centroid(county, id_col, geo_col)
        print(f'Centroid located at {centroid}.')
        weather = create_met(centroid[0], centroid[1], 1980, 2019, county_name)
        print(f"Met file for {county_name}/{i} at location {centroid} created.")

### Step 3: Get all distinct counties and create met files for all county geometry lat/lon

In [ ]:
#fips = get_distinct(dbconn, raccoon_2018, 'fips')
#create_all_met(dbconn, fips, raccoon_2018)

### Step 4: Get clukey crop rotations

In [ ]:
"""
Get the crop rotation for each clukey.

Args:
    df {obj} -- Dataframe that contains individual clukey information.
    crop_column {str} -- Column name that contains the label for what crop is growing for a given year.

Returns:
    Str of the rotation. e.g., 'cs' = corn-soy
"""
def get_rotation(df, crop_column):
    #save rotation for clukey to crops list
    crops = []
    for i in df.index:
        val = df.loc[i, crop_column]
        if val == 'Corn' or val == 'Soybean':
            crops.append(val)
        else:
            crops.append('other')
    #evaluate crops list and return a rotation
    if all(x in crops for x in ['Corn', 'Soybean']):
        rotation = 'cs'
    elif all(x in crops for x in ['Corn']):
        rotation = 'cc'
    else:
        rotation = 'other'
    return rotation

In [ ]:
rots = pd.read_sql('SELECT * FROM raccoon.raccoon_rots', dbconn)
rots['rotation'] = ''

In [ ]:
#group df by clukey, sort by year, then loop through and get rotation, appending to original df.
grouped = rots.groupby('clukey')
clukeys = rots.drop_duplicates('clukey')
for i in clukeys['clukey']:
    field = grouped.get_group(i).sort_values(by=['years'], ascending=True)
    rotation = get_rotation(field, 'crop')
    rots.loc[rots['clukey'] == i, 'rotation'] = rotation

In [ ]:
rots.head()

### Step 5: Get the soil properties for each individual mukey

In [ ]:
#def create_input_table(dbconn, table, fip, start_year=2016, end_year=2018, id_col='fips', geo_col='wkb_geometry', name_col='county', soil_col="mukey", limit=False):
def create_apsim_files(df, rotations_df, dbconn, field_key='clukey', soil_key='mukey', county_col='county', rotation_col='rotation', crop_col='crop', start_year=2016, end_year=2018):
    if not os.path.exists('apsim_files'):
        os.makedirs('apsim_files')
    start_date = f'01/01/{start_year}'
    end_date = f'31/12/{end_year}'
    #save rotation for clukey to crops list
    df = df.drop_duplicates(field_key)
    #loop through field keys e.g., clukeys
    for i in df[field_key]:
        field_id = i
        #get field information
        #TODO get 'clukey' and 'county' to work as function inputs instead of hardcoded
        field = df.loc[df['clukey'] == i]
        #get field rotation
        rotation_row = rotations_df.loc[rotations_df[field_key] == i]
        rotation = get_rotation(rotation_row, crop_col)
        #get unique soil keys e.g., mukeys
        soils = field.drop_duplicates(soil_key)
        #get weather file for desired county
        county_name = field.iloc[0]['county'].replace(" ", "_")
        met_name = f"{county_name}.met"
        met_path = f"met_files/{met_name}"
        #create apsim file for each unique soil in field
        for i in soils[soil_key]:
            try:
                soil_id = i
                soil_query = '''select * from api.get_soil_properties( array[{}]::text[] )'''.format( i )
                soil_df = pd.read_sql( soil_query, dbconn )
                if soil_df.empty:
                    continue
                #soil_row = soils_df.loc[soils_df[f'{soil_key}'] == i]
                #initialize .apsim xml
                apsim_xml = Element( 'folder' )
                apsim_xml.set( 'version', '36' )
                apsim_xml.set( 'creator', 'C-CHANGE Foresite' )
                apsim_xml.set( 'name', county_name )
                sim = SubElement( apsim_xml, 'simulation' )
                sim.set( 'name', f'{county_name} {field_id}' )
                
                #set met file
                metfile = SubElement( sim, 'metfile' )
                metfile.set( 'name', f'{county_name}' )
                filename = SubElement( metfile, 'filename' )
                filename.set( 'name', 'filename' )
                filename.set( 'input', 'yes' )
                filename.text = met_path

                #set clock
                clock = SubElement( sim, 'clock' )
                clock_start = SubElement( clock, 'start_date' )
                clock_start.set( 'type', 'date' )
                clock_start.set( 'description', 'Enter the start date of the simulation' )
                clock_start.text = start_date
                clock_end = SubElement( clock, 'end_date' )
                clock_end.set( 'type', 'date' )
                clock_end.set( 'description', 'Enter the end date of the simulation' )
                clock_end.text = end_date
                sumfile = SubElement( sim, 'summaryfile' )
                area = SubElement( sim, 'area' )
                area.set( 'name', 'paddock' )

                # add soil xml
                soil = apsim.Soil( soil_df, SWIM = False, SaxtonRawls = False )
                area.append( soil.soil_xml() )
                ### surface om
                surfom_xml = apsim.init_surfaceOM( 'maize', 'maize', 3500, 65, 0.0 )
                area.append( surfom_xml )
                ### fertilizer
                fert_xml = SubElement( area, 'fertiliser' )

                ### crops
                crop_xml = SubElement( area, 'maize' )
                crop_xml = SubElement( area, 'soybean' )
                crop_xml = SubElement( area, 'wheat' )

                ### output file
                outvars = [
                    'dd/mm/yyyy as Date', 'day', 'year',
                    'yield', 'biomass', 'fertiliser',
                    'surfaceom_c', 'subsurface_drain',
                    'subsurface_drain_no3', 'leach_no3',
                    'corn_buac', 'soy_buac' ]
                output_xml = apsim.set_output_variables( f'{county_name}_{field_id}_{soil_id}.out', outvars )
                area.append( output_xml )

                graph_no3 = [
                    'Cumulative subsurface_drain',
                    'Cumulative subsurface_drain_no3',
                    'Cumulative leach_no3'
                ]
                graph_yield = [
                    'yield',
                    'biomass',
                    'corn_buac'
                ]
                graph_all = [
                    'yield', 'biomass', 'fertiliser',
                    'surfaceom_c', 'Cumulative subsurface_drain',
                    'Cumulative subsurface_drain_no3',
                    'Cumulative leach_no3', 'corn_buac',
                    'soy_buac'
                ]

                output_xml.append( apsim.add_xy_graph( 'Date', graph_no3, 'no3' ) )
                output_xml.append( apsim.add_xy_graph( 'Date', graph_yield, 'yield' ) )
                output_xml.append( apsim.add_xy_graph( 'Date', graph_all, 'all outputs' ) )

                op_man = apsim.OpManager()
                op_man.add_empty_manager()
                if rotation == 'cs':
                    add_management_year(op_man, spin_up_corn, 2016)
                    add_management_year(op_man, spin_up_soybean, 2017)
                    add_management_year(op_man, spin_up_corn, 2018)
                elif rotation == 'cc':
                    add_management_year(op_man, spin_up_corn, 2018)
                    add_management_year(op_man, spin_up_corn, 2016)
                    add_management_year(op_man, spin_up_corn, 2017)
                area.append( op_man.man_xml )
                outfile = f'apsim_files/{county_name}_{field_id}_{soil_id}.apsim'
                ### management data
                tree = ElementTree()
                tree._setroot( apsim_xml )
                tree.write( outfile )
            except (RuntimeError, TypeError, NameError):
                continue

In [ ]:
create_apsim_files(greene, rots, dbconn)

In [ ]:
#grouped.first()
# field = grouped.get_group(25197).sort_values(by=['years'], ascending=True)
# racc_field = raccoon.loc[raccoon['clukey'] == 2539919]
#soil_row = soil_df.loc[soil_df['mukey'] == '410373']
#soil_row['claytotal_r'][1]
#soil_row.head()
# racc_soils = racc_field.drop_duplicates('mukey')
# for i in racc_soils['mukey']:
#     soil = soil_df.loc[soil_df['mukey'] == i]
#     #print(soil)
# rotation_row = rots.loc[rots['clukey'] == 2539919]
# rotation = rotation_row['rotation'][1]